# План семинара

- Случайные величины. Нормальный закон распределения.
    - Распределение вероятностей
    - Функция распределения
    - Нормальный закон распределения
        - Закон больших чисел и центральная предельная теорема
        - Формулы для функции и плотности нормального распределения
        - Генерирование выборки, графики функций распределения и плотностей, эмпирических и теоретических
    - Нахождение доверительных интервалов
        - Доверительный интервал на основе нормального распределения
        - Доверительный интервал Уилсона (лучше, для средних значений близких к 0 и 1)
- Проверка гипотез
    - Критерии Стьюдента
        - Одновыборочный критерий Стьюдента
        - Двувыборочный критерий Стьюдента
        - Двувыборочный критерий Стьюдента для зависимых выборок
    - Непараметрические криетрии
        - Ранговый критерий Манна-Уитни
- A/B тестирование

In [ ]:
import pandas as pd
import numpy as np
import random
import scipy as sp
import scipy.stats as sts
import matplotlib.pyplot as plt
%matplotlib inline

random.seed(42)

<a id='rv'></a>
# Случайные величины. Нормальный закон распределения.

Случайные величины бывают:
- дискретные
- непрерывные

На практике статистические заключения опираются на допущения о том, как распределены рассматриваемые случайные переменные (данные). (Даже принято преобразовывать данные с тем, чтобы они соответствовали одному из известных распределений)

<a id='pd'></a>
## Распределение вероятностей

*Закон распределения вероятностей* — это выражение, которое определяет, какие значения будет принимать данная переменная или параметр, и как часто будет встречаться каждое из этих значений.

Иногда закон распределения случайной величины можно выписать сразу. Например, если случайная величина $X$ - число выпадающих  очков при бросках игральной кости (шестигранный кубик с точками), то закон распределения ее значений такой:

| | | | | | | |
|-------------:|:---------|:--------|:---------|:---------|:--------|:---------|
|$$X$$         |1         |2        |3         |4         |5        |6         |
|$$p(X)$$      |1/6       |1/6      |1/6       |1/6       |1/6      |1/6       |


Но далеко не всегда мы  знаем заранее закон распределения случайной величины (ее функцию вероятности). В таком случае мы ее стараемся оценить на основании имеющихся данных. 

Раздобудем результаты $n=200$ бросков игральной кости. Чтобы не бросать кубик самим, смоделируем результаты в Python:

In [ ]:
n=20000
sample = np.random.choice([1,2,3,4,5,6], n)
print("Выпавшие значения Х:",sample )

Тогда вероятности значений случайной величины  X можно оценить с помощью *относительных частот* выпадения значений. 

In [ ]:
# посчитаем число выпадений каждой из сторон:
from collections import Counter

c = Counter()
for i in sample:
    c[i] += 1

print("Число выпадений каждого значения:")    
print(c, '\n')

# получим вероятности, поделив каждое число выпадений на общее количество бросков:

print("Вероятности выпадений каждого значения:")
p_X={key: value/n for key, value in c.items()}
print(p_X)

In [ ]:
# Нарисуем распределение вероятностей:
for key in p_X:
    plt.scatter([key], p_X[key], label=key)
plt.show()

**Задание.** Проведите эксперименты для $n=2000,20000,200000$ бросков. Что происходит с вероятностями? Что это за значение, к которому вероятности стремятся?

<a id='cdf'></a>
## Функция распределения

Знание распределения вероятностей позволяет построить *функцию распределения* $F(x)$. (Обратное тоже верно)))

$F(x)$ - это вероятность того, что случайная величина  $X$  примет значение меньшее, чем $x$, т.е.

$$F(x)=P(X<=x)$$

Из этого определения следует очень важная для практики формула вероятности попадания случайной величины $X$ в интервал:

**$$P(a<X<=b)=F(b)-F(a)$$**



In [ ]:
sample.shape

In [ ]:
# Построим функцию распределения для нашей выборки. 
from statsmodels.distributions.empirical_distribution import ECDF
F = ECDF(sample, side='right')
plt.step(F.x, F.y, label='F(x)')

plt.ylabel('$F(x)$')
plt.xlabel('$x$')
plt.legend(loc='upper left')

In [ ]:
F([5]), F([10])

**Задание.**
1. Найдите F(1), F(3), F(6), F(10). Почему получились именно такие значения?

2. Найдите вероятность того, что при броске выпадет не менее 3 и не более 5 очков.

In [ ]:
# your code here

Построим гистаграмму принимаемых значений

In [ ]:
plt.hist(sample, bins=6, density=True);

In [ ]:
plt.hist(sample, bins=100, density=True, cumulative=True);

<a id='nb'></a>
## Нормальный закон распределения

В теории статистики доказаны две теоремы, которые обосновывают все статистические вычисления. В силу своей важности эти теоремы получили громкие названия - Центральная предельная теорема и Закон больших чисел.



<a id='lbn'></a>
### Закон больших чисел и центральная предельная теорема

*Закон больших чисел*  утверждает, что какое бы ни было распределение случайной величины, среднее арифметическое ее реализаций при их большом числе близко к истинному значению среднего значения распределения. 

*Центральная предельная теорема* утверждает, что если случайная величина формирует свои значения под действием многочисленных, независимых факторов, влияние каждого из которых пренебрежимо мало, то такая величина распределена нормально. 

То есть имеет распределение, которое называют *нормальным*, и свойства которого очень хорошо изучены. 

По ЦПТ получается, что среднее случайных величин будет распределено как $N(\mu, \sigma^2/n)$

Напишите функцию, которая будет выдавать ожидаемое нами стандартное отклонение для среднего по k случайных равномерно распределенным  от 0 до 1 величинам

*Подсказка:*
Вообще говоря дисперсия равномерной случайной величины равна $\frac{(b - a)^2}{12}$

In [ ]:
def get_std(k):
    return # your code here

#### Давайте посмотрим на теорему в действии

Сформируем случайную величину, которая представляет собой сумму $k$ случайных чисел из какого-нибудь распределения (например, равномерного). Сгенерируем  $N$ таких сумм и посмотрим, как они будут распределены.

In [ ]:
N = 10000
k = 10

In [ ]:
# Убедимся, что умеем генерировать массив из N случайных равномерно распределенных на [0; 1] чисел
plt.hist(np.random.random(N));

In [ ]:
# Складываем покоординатно k массивов, получаем массив из N сумм, строим  гистограмму
uniform_sum = sum(np.random.random(N) for _ in range(k))
plt.hist(uniform_sum, bins=100);

In [ ]:
# А теперь давайте посмотрим на то, как распределено среднее по этим величинам
uniform_mean = uniform_sum / k
plt.hist(uniform_mean, bins=100)
plt.xlim([0,1]);

In [ ]:
np.mean(uniform_mean), np.std(uniform_mean), get_std(k)

In [ ]:
k = 100

In [ ]:
uniform_sum = sum(np.random.random(N) for _ in range(k))
uniform_mean = uniform_sum / k
plt.hist(uniform_mean, bins=100)
plt.xlim([0,1]);

In [ ]:
np.mean(uniform_mean), np.std(uniform_mean), get_std(k)

In [ ]:
k = 1000

In [ ]:
uniform_sum = sum(np.random.random(N) for _ in range(k))
uniform_mean = uniform_sum / k
plt.hist(uniform_mean, bins=100)
plt.xlim([0,1]);

In [ ]:
np.mean(uniform_mean), np.std(uniform_mean), get_std(k)

Эмпирический результат совпал с теоретическим -- ура!

<a id='pdf'></a>
### Формулы для функции и плотности нормального распределения

*Теория.  Взглянуть мельком.*

Функция нормального распределения:

$$Ф(x)=\frac{1}{\sigma\sqrt{2\pi}}\int\limits_{-\infty}^x\exp\left(-\frac{(t-\mu)^2}{2\sigma^2}\right) dt. $$

Плотность нормального распределения

 $$  f(x) = \frac{1}{\sigma\sqrt{2\pi}}\exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right). $$

<a id='gr'></a>
### Генерирование выборки, графики функций распределения и плотностей, эмпирических и теоретических

In [ ]:
mu=0
sigma=1
n = 100

# Сгенерируем выборку из нормального распределения
sample_normal = np.random.normal(mu, sigma, n)

# 10 первых значений
sample_normal[:10]

In [ ]:
norm_random_variates = sts.norm(mu, sigma)

In [ ]:
# Нарисуем функцию распределения: теоретическую и эмпирическую, составленную по выборке

# теоретическая функция распределения cdf - cumulative density function
x = np.linspace(-5,5, 100)
cdf = norm_random_variates.cdf(x)
plt.plot(x, cdf, label='теоретическая Ф(x) ')

# 'эмпирическая функция распределения
F = ECDF(sample_normal)
plt.step(F.x,F.y, label='эмпирическая, F(x)')

plt.ylabel('$Ф(x)$')
plt.xlabel('$x$')
plt.legend(loc='upper left')

In [ ]:
plt.hist(sample_normal, bins=20, density=True)
plt.ylabel('частота встречаемости значений')
plt.xlabel('$x$')

#### Возьмем n побольше

In [ ]:
n = 100000
# Сгенерируем выборку из нормального распределения
sample_normal = np.random.normal(mu, sigma, n)

In [ ]:
# теоретическая функция распределения cdf - cumulative density function
x = np.linspace(-5,5, 100)
cdf = norm_random_variates.cdf(x)
plt.plot(x, cdf, label='теоретическая Ф(x) ')

# 'эмпирическая функция распределения
F = ECDF(sample_normal)
plt.step(F.x,F.y, label='эмпирическая, F(x)')

plt.ylabel('$Ф(x)$')
plt.xlabel('$x$')
plt.legend(loc='upper left')

In [ ]:
# Построим гистограмму
plt.hist(sample_normal, bins=100, density=True)
plt.ylabel('частота встречаемости значений')
plt.xlabel('$x$')

*Плотностью распределения* называется производная от функции распределения: $f(x)=F'(x)$

In [ ]:
# Нарисуем плотность распределения: теоретическую и эмпирическую, составленную по выборке

# эмпирическая, построенная по выборке с помощью ядерного сглаживания - KDE, Kernel Density Estimation 
# используем библиотеку Pandas:
df = pd.DataFrame(sample_normal, columns=['KDE'])
ax = df.plot(kind='density')

# теоретическая плотность - pdf probability density function
x = np.linspace(-5,5,100)
pdf = norm_random_variates.pdf(x)
plt.plot(x, pdf, label='теоретическая плотность распределения', alpha=0.5)
plt.legend()
plt.ylabel('$f(x)$')
plt.xlabel('$x$')

<a id='ci'></a>
## Нахождение доверительных интервалов

Предварительно сделаем несколько упражнений

*Упр. 1*

Найдем, левее какого числа находится 95% значений нормально распределенной случайной величины с $\mu=0$ и $\sigma=1$, т.е. 95-ю квантиль

In [ ]:
mu=0
sigma=1
sts.norm(mu, sigma).ppf(0.95)

*Упр. 2*

Выполним обратную операцию, убедимся, что вероятность принять значения, меньшие чем 1.6448536269514722, равна 95%


In [ ]:
sts.norm(mu, sigma).cdf(1.644853626951472)

*Упр. 3*

Пользуясь формулой $P(a\le X<b)=F(b)-F(a)$ определим вероятность того, что нормально распределенная случайная величина с $\mu=0$  $\sigma=1$ примет значения $x \in [2;3]$ 

In [ ]:
P=sts.norm(mu, sigma).cdf(3)-sts.norm(mu, sigma).cdf(2)
print(P)

На практике приходится решать и обратную задачу - находить, в каких границах  могут находиться с заданной вероятностью ("надежностью") значения случайной величины.

**Доверительным** называют интервал, который покрывает неизвестный параметр с заданной надёжностью.

В дальнейшем нам нужно будет находить доверительный интервал для математического ожидания  нормально распределенной случайной величины с надежностью $\gamma$. 

*Теория. *


В теории при небольших объемах данных такой интервал можно найти из формулы:
$${P}\left( {\bar X}-t_{\frac{1+\gamma}{2},n-1}\cdot \frac{S}{\sqrt{n}}\le \mu\le {\bar X}+t_{\frac{1+\gamma}{2},n-1}\cdot \frac{S}{\sqrt{n}}\right)=\gamma, $$
где $t_{\alpha,n-1}$ -- $\alpha$-квантили распределения Стьюдента, т.е. распределения случайой величины $T=\frac{{\bar X}-\mu}{S\,/\,\sqrt{n}}$, а $S$ -- несмещеное выборочное стандартное отклонение.

При больших объемах и/или известном стандартном отклонении математического ожидания вместо распределения Стьюдента используется нормальное распределение, и

$${P}\left( {\bar X}-z_{\frac{1+\gamma}{2}}\cdot \frac{\sigma}{\sqrt{n}}\le \mu\le {\bar X}+z_{\frac{1+\gamma}{2}}\cdot \frac{\sigma}{\sqrt{n}}\right)=\gamma, $$
где $z_{\alpha}$ -- $\alpha$-квантили стандартного нормального распределения, т.е. распределения случайной величины $Z=\frac{{\bar X}-\mu}{\sigma\,/\,\sqrt{n}}$, а $S$ -- известное стандартное отклонение.

Возьмем выборку из распределения бернулли

In [ ]:
random_ber = np.random.randint(2, size=50)

#### Доверительный интервал на основе нормального распределения

$$\hat{p}\pm z_{1-\frac{\alpha}{2}} \sqrt{\frac{\hat{p}\left(1-\hat{p}\right)}{n}}$$

In [ ]:
from statsmodels.stats.proportion import proportion_confint

In [ ]:
def normal_int(random_ber):
    
    s = random_ber.sum()
    n = len(random_ber)
    
    normal_interval = proportion_confint(s, n, method = 'normal', alpha = 0.05)
    return np.array(normal_interval)


def normal_int_custom(random_ber):
    
    s = random_ber.sum()
    n = len(random_ber)
    z_alpha = 1.959964
    
    p_hat = # your code here
    half_int = # your code here
    return # your code here

In [ ]:
assert np.allclose(normal_int(random_ber), normal_int_custom(random_ber))

In [ ]:
for size in [10, 100, 1000, 10000]:
    random_ber = np.random.randint(2, size=size)
    print(normal_int(random_ber))

#### Доверительный интервал Уилсона (лучше, для средних значений близких к 0 и 1)

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{
z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [ ]:
p = 0.9
random_ber = np.random.choice(2, size=1000, p=[1.-p, p])

In [ ]:
def wilson_int(random_ber):
    
    s = random_ber.sum()
    n = len(random_ber)
    
    normal_interval = proportion_confint(s, n, method = 'wilson', alpha = 0.05)
    return np.array(normal_interval)

def wilson_int_custom(random_ber):
    
    s = random_ber.sum()
    n = len(random_ber)
    z_alpha = 1.959964
    
    p_hat = # your code here
    
    half_int = # your code here
    
    return 1/(1 + z_alpha*z_alpha/n)*np.array([p_hat + z_alpha**2/(2*n) - half_int, 
                                               p_hat + z_alpha**2/(2*n) + half_int])

In [ ]:
print(wilson_int(random_ber))
print(wilson_int_custom(random_ber))

In [ ]:
assert np.allclose(wilson_int(random_ber), wilson_int_custom(random_ber), atol=1e-4)

In [ ]:
p = 0.9

for size in [10, 100, 1000, 10000]:
    random_ber = np.random.choice(2, size=size, p=[1.-p, p])
    print(wilson_int(random_ber), normal_int(random_ber))

[Прикольная визуализация](https://rpsychologist.com/d3/ci/), если еще хочется посмотреть на то, что такое доверительный интервал

# Проверка гипотез

P-value
![title](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/P-value_in_statistical_significance_testing.svg/474px-P-value_in_statistical_significance_testing.svg.png)

### Критерии Стьюдента (T-тест)

Работает для тех значений, которые распределены нормально

Нужно проверить на нормальность:
- q-q plot
- Критерий Шапиро-Уилка: (гипотеза о нормальности распределения)

In [ ]:
from scipy import stats
%pylab inline

In [ ]:
test_norm = np.random.normal(size=5000)
test_exp = np.random.exponential(size=5000)

In [ ]:
plt.hist(test_norm, bins=100)
plt.show()

In [ ]:
plt.hist(test_exp, bins=100)
plt.show()

In [ ]:
stats.probplot(test_norm, dist ="norm", plot = pylab)
pylab.show()

In [ ]:
stats.probplot(test_exp, dist="norm", plot = pylab)
pylab.show()

In [ ]:
print("Shapiro-Wilk normality test, W-statistic: %f, p-value: %f" % stats.shapiro(test_norm))

In [ ]:
print("Shapiro-Wilk normality test, W-statistic: %f, p-value: %f" % stats.shapiro(test_exp))

### а) Одновыборочный критерий Стьюдента

$H_0\colon$ среднее значение некоторой выборки равняется заданному числу $m$,

$H_1\colon$ не равно.

Критерий:

$$t = \frac{\bar{X} - m}{s/\sqrt{n}}$$ 

$$s^2 = \frac{1}{n-1}\sum_{i}^{n} (X_i - \bar{X})^2$$ 

где $\bar{X}$ - среднее значение случайной величины $X$; $s$ - несмещенная оценка дисперсии; $n$ - размер выборки. При нулевой гипотезе эта статистика имеет распределение Стьюдента с $(n-1)$ степенями свободы.

Для полученного значения критерия вычисляем p-value $p$. 
- Если $p < \alpha$, то гипотеза $H_0$ отвергается в пользу $H_1$.
- Если $p \ge \alpha$, то принимается гипотеза $H_0$.

Обычно пороговое значение $\alpha=0.05$.

In [ ]:
import scipy
from statsmodels.stats.weightstats import *

stats.ttest_1samp(test_norm, 0.02)

In [ ]:
print("95%% confidence interval: [%f, %f]" % zconfint(test_norm))

In [ ]:
stats.ttest_1samp(test_norm, 0.05)

### б) Двувыборочный критерий Стьюдента

$H_0\colon$ средние значения двух выборок одинаковы

$H_1\colon$ не одинаковы.

Рассмотрим разность средних значений двух выборок $\bar{X}_1$ и $\bar{X}_2$:

$$\Delta = \bar{X}_1 - \bar{X}_2$$

Дисперсия разности равна:

$$s^2_{\Delta} = \frac{s_1^{2}}{n_1} + \frac{s_2^{2}}{n_2}$$ 

где

$$s^2 = \frac{1}{n-1}\sum_{i}^{n} (X_i - \bar{X})^2$$ 


Двувыборочный критерий Стьюдента:

$$t = \frac{\Delta}{s_{\Delta}}$$ 


где $\bar{X}$ - среднее значение случайной величины $X$; $s$ - несмещенная оценка дисперсии; $n$ - размер выборки. При нулевой гипотезе  и $s_1 = s_2$ эта статистика имеет распределение Стьюдента с $(n_1 + n_2 - 2)$ степенями свободы.

Для полученного значения критерия вычисляем p-value $p$. 
- Если $p < \alpha$, то гипотеза $H_0$ отвергается в пользу $H_1$.
- Если $p \ge \alpha$, то принимается гипотеза $H_0$.

Обычно пороговое значение $\alpha=0.05$.

In [ ]:
test_a = np.random.normal(loc = 0.05, size=10000)
test_b = np.random.normal(loc = 0.0, size=10000)

In [ ]:
plt.hist(test_a, bins=100, alpha = 0.3)
plt.hist(test_b, bins=100, alpha = 0.3)
plt.show()

In [ ]:
print("95%% confidence interval for a: [%f, %f]" % zconfint(test_a))
print("95%% confidence interval for b: [%f, %f]" % zconfint(test_b))

In [ ]:
scipy.stats.ttest_ind(test_a, test_b, equal_var=False)

### d) Двувыборочный критерий Стьюдента для зависимых выборок
$H_0\colon$ средние значения двух выборок одинаковы

$H_1\colon$ не одинаковы.

Критерий:

$$t = \frac{M_d}{s_d/\sqrt{n}}$$ 

где $M_d$ - среднее значение случайной величины $d = X_1 - X_2$; $s_d$ - несмещенная оценка дисперсии это случайной величины; $n$ - размер выборки. При нулевой гипотезе эта статистика имеет распределение Стьюдента с $(n-1)$ степенями свободы.

Для полученного значения критерия вычисляем p-value $p$. 
- Если $p < \alpha$, то гипотеза $H_0$ отвергается в пользу $H_1$.
- Если $p \ge \alpha$, то принимается гипотеза $H_0$.

Обычно пороговое значение $\alpha=0.05$.

In [ ]:
SIZE = 1000

In [ ]:
test_ab = np.random.normal(size=SIZE)
test_a = test_ab + np.random.normal(loc=0.4, size=SIZE)
test_b = test_ab + np.random.normal(loc=0.5, size=SIZE)

In [ ]:
mean_list = test_a - test_b
plt.hist(mean_list, bins=100, alpha = 0.3)
plt.show()

In [ ]:
scipy.stats.ttest_ind(test_a, test_b, equal_var=False)

In [ ]:
stats.ttest_rel(test_a, test_b)

In [ ]:
print("95%% confidence interval: [%f, %f]" % DescrStatsW(mean_list).tconfint_mean())

# Непараметрические криетрии

Используются, если выборки распределены ненормально

### Двувыборочный непараметрический критерий для независимых выборок
$H_0\colon$ средние значения двух выборок одинаковы

$H_1\colon$  не одинаковы.

### Ранговый критерий Манна-Уитни

- Собираем наблюдения в одну выборку с $n = n_1 + n_2$ наблюдениями.
- Ранжируем наблюдения. Считаем сумму рангов отдельно для каждой группы: $R_x$ и $R_y$.
- Считаем статистики:

$$U_x = R_x - \frac{n_1(n_1+1)}{2}$$
$$U_y = R_y - \frac{n_2(n_2+1)}{2}$$
$$U = \min(U_x, U_y)$$

- Для полученного значения критерия вычисляем p-value $p$. 
    - Если $p < \alpha$, то гипотеза $H_0$ отвергается в пользу $H_1$.
    - Если $p \ge \alpha$, то принимается гипотеза $H_0$.

Обычно пороговое значение $\alpha=0.05$.

In [ ]:
np.random.seed(42)
N = 10

test_a = np.random.beta(2., 2., size=N)
test_b = np.random.beta(2., 3., size=N)

In [ ]:
test_a.mean(), test_b.mean()

In [ ]:
plt.hist(test_a, bins=100, alpha=0.6)
plt.hist(test_b, bins=100, alpha=0.6)
plt.show()

In [ ]:
scipy.stats.ttest_ind(test_a, test_b, equal_var=False)

In [ ]:
stats.mannwhitneyu(test_a, test_b)

In [ ]:
print('95%% confidence interval for the mean test_a: [%f, %f]' % zconfint(test_a))
print('95%% confidence interval for the mean test_b: [%f, %f]' % zconfint(test_b))


<a id='ab'></a>
##  A/B тестирование

A/B-тестирование (англ. A/B testing, Split testing) — метод маркетингового исследования, суть которого заключается в том, что контрольная группа элементов сравнивается с набором тестовых групп, в которых один или несколько показателей были изменены, для того, чтобы выяснить, какие из изменений улучшают целевой показатель и улучшают ли.

Типичное применение в веб-дизайне — исследование влияния цветовой схемы, расположения и размера элементов интерфейса на конверсию сайта.

Конверсия (Conversion Rate) в интернет-маркетинге — это отношение числа посетителей сайта, выполнивших на нём какие-либо целевые действия (покупку, регистрацию, подписку, посещение определённой страницы сайта, переход по рекламной ссылке), к общему числу посетителей сайта, выраженное в процентах. 



В теории принципы A/B тестирования невероятно просты:

- Выдвигаем предположение о том, что какое-то изменение (например, персонализация главной страницы) увеличит конверсию интернет-магазина.

- Создаем альтернативную версию сайта «Б» — копию исходной версии «А» с изменениями, от которых мы ждем роста эффективности сайта.

- Всех посетителей сайта случайным образом делим на две равные группы: одной группе показываем исходный вариант (контрольная группа) , второй группе (тестовой) — альтернативный. Одновременно измеряем конверсию для обеих версий сайта.

- Определяем статистически достоверно победивший вариант.

Мы будем анализировать результаты A/B тестирования двух версий дизайна кнопки сайта интернет-магазина. 

Целевым действием считаем клик по этой кнопке. 

Первые три этапа А/В тестирования за нас провели, результаты предоставили в виде файла ab_dataset.csv. Нам осталось выполнить четвертый пункт.

 <a id='eda'></a>
### Первичный анализ данных

Прочитаем данные из файла `ab_dataset.csv`. Сохраним их в датафрейм `df`. 

Прочитаем данные и посмотрим на первые 5 строк:

In [ ]:
!wget https://raw.githubusercontent.com/hse-ds/iad-applied-ds/master/2021/seminars/sem10_stat/ab_data.csv

In [ ]:
df = pd.read_csv('ab_data.csv')

df.head()

Посмотрим, сколько посетителей заходио на сайт (количество строк в нашем датафрейме)

In [ ]:
#количество посетителей всего

n_rows = df.shape[0]
print("Число строк: {}".format(n_rows))

Сколько уникальных пользователей (уникальных `user_id`) в датасете?

In [ ]:
user_total = df.nunique()['user_id']
print("Число уникальных пользователей : {}".format(user_total))

Посетителей из контрольной `control` группы должны были направлять на страницу в старом дизайне  `old_page` , пользователей из тестовой группы `treatment` - на страницу в новом дизайне `new_page`. Проверим, были ли ошибки при направлении.

In [ ]:
mismatch_1 = df.query("group == 'treatment' and landing_page == 'old_page'")
print("Из тестовой группы неверно направлены {}".format(len(mismatch_1)) + " пользователей")

mismatch_2 = df.query("group == 'control' and landing_page == 'new_page'")
print("Из контрольной группы неверно направлены  {}".format(len(mismatch_2)) + " пользователей")


Выясним, есть ли в данных пропуски.

In [ ]:
df.info()

Из сообщений следует, что пропущенных значений нет.

Конверсия по всем посетителям

In [ ]:
p_all=df['converted'].mean()
print("Конверсия по всем посетителям: {} %".format(p_all*100))

In [ ]:
# можно и так:
sum(df['converted'].values)/n_rows

Давайте посмотрим на описательную статистику нашего датасета (воспользуемся функцией  `describe`) и постараемся получить ответы на следующие вопросы:

- Какова вероятность клика для посетителей из контрольной группы (старый дизайн)?

- Какова вероятность клика для посетителей из тестовой группы (новый дизайн кнопки)?

- Каково соотношение размеров тестовой и контрольный групп? Какова вероятность, что очередной посетитель будет направлен на версию со старым дизайном? С новым дизайном?

In [ ]:
df_grp = df.groupby('group')
df_grp.describe()

Ответы на остальные вопросы можно найти, например, так:

In [ ]:
#объем тестовой группы
n_rows_treat = len(df[df['group'] == 'treatment'])

#объем контрольной группы
n_rows_contr = n_rows-n_rows_treat

print("Соотношение размеров тестовой и контрольной групп: {}".format(n_rows_treat/n_rows_contr))

print("Вероятность, что новый пользователь будет направлен на версию со старым дизайном: {}".format(n_rows_treat/n_rows))
print("Вероятность, что новый пользователь будет направлен на версию с новым дизайном: {}".format(n_rows_contr/n_rows))


**Задание **

А теперь ответьте на главный вопрос данного этапа:  выявил ли предварительный анализ, что дизайн кнопки влияет на конверсию и если да, то как именно?

<a id='si'></a>
### Статистический вывод для A/B Теста

По нашим данным очевидно, что кликабельность старой версии выше, чем новой, и напрашивается решение оставить всё как есть.  Однако бизнес потребует обоснований для такого глобального решения, бОльших, чем "очевидно по результатам одного эксперимента". 

И такие обоснования предоставляет специально разработанная методология - статистический вывод. 

Статистический вывод - это переход от данных о статистической выборке (нашего датасета) к обобщениям в виде параметров генеральной совокупности с вычислением степени уверенности в справедливости этих обобщений.

Будем считать, что клик — это некоторая случайная переменная , принимающая значения 1  или 0 с вероятностями $\theta$ и $1-\theta$ соответственно. 

Применительно к нашей задаче посетитель может кликнуть на кнопку (с вероятностью $\theta$) или не кликнуть на нее (с вероятностью, соответственно,  $1-\theta$)

На практике статистические заключения опираются на допущения о том, как распределены рассматриваемые случайные переменные (данные). (Даже принято преобразовывать данные с тем, чтобы они соответствовали одному из известных распределений)

Распределение вероятностей — это выражение, которое определяет, какие значения будет принимать данная переменная или параметр,  и как часто будет встречаться каждое из этих значений.

 Наша случайная переменная — клик — имеет распределение, которое называется распределением Бернулли с параметром $\theta$:
$$ p(k) = \theta^k (1 - \theta)^{1 - k}, $$

где $p(k)$ —  вероятность случайной переменной  принять значение $k$, $k \in \{ 0;1\}$.

Из теории известно, что среднее значение распределения Бернулли равно $\mu = \theta$, а дисперсия равна $\sigma^2 = \theta(1 - \theta)$. Нас интересует конверсия сайта, в рассматриваемой постановке она равна $\theta$.

<a id='si1'></a>
#### Формулировка рабочей гипотезы

Рабочая гипотеза в нашем случае может быть сформулирована, например, так: "Конверсия сайта со старым дизайном не меньше, чем с новым"

<a id='si2'></a>
#### Формальное описание нулевой и альтернативной гипотез

$H_{0}$ : $\theta_{new}$ <= $\theta_{old}$ 

$H_{1}$ : $\theta_{new}$ > $\theta_{old}$ 

<a id='si1'></a>
#### Выбор подходящего статистического теста (статистического критерия)

Истинные значения $\theta_{new}$ и $\theta_{old}$ мы найти не можем, но мы можем их сравнить. В этом помогут те самые две теоремы.






На основании Закона больших чисел мы можем оценить средние значения случайных величин $\theta_{new}$ и $\theta_{old}$ на  генеральных совокупностях по их средним значениям на наших больших выборках.

На основании Центральной предельной теоремы средние значения распределены нормально при больших объемах выборок.

Чтобы выбрать для проверки наших гипотез подходящий статистический критерий, уясним задачу. Нам нужно выяснить, случайно или значимо отличаются средние значения, т.е. доли, кликнувших посетителей в контрольной и тестовой выборках.

<a id='si4'></a>
#### Проведение вычислений. $Z$-критерий.

$$Z = \frac{p_1 - p_2}{\sqrt{p(1-p) (\frac{1}{n_1} + \frac{1}{n_2})}}$$

где $p_1$ - конверсия в первой группе; $p_2$ - конверсия во второй группе; $p$ - конверсия во всех наблюдениях; $n_1$, $n_2$ - количество наблюдений в группах.

Для полученного значения критерия вычисляем p-value $p$. 
- Если $p < \alpha$, то гипотеза $H_0$ отвергается в пользу $H_1$.
- Если $p \ge \alpha$, то принимается гипотеза $H_0$.

Обычно пороговое значение $\alpha=0.05$.

In [ ]:
#Z-статистика
import statsmodels.api as sm
import statsmodels

convert_contr = sum(df.query("group == 'control'")['converted'])
convert_treat = sum(df.query("group == 'treatment'")['converted'])

z_score, p_value = sm.stats.proportions_ztest([convert_treat, convert_contr], [n_rows_treat, n_rows_contr], 
                                              alternative='larger')
print("Z-статистика={},  p_value={}".format(z_score, p_value))


#### Проведение вычислений. $T$-test.

In [ ]:
t_score, p_value, _ = statsmodels.stats.weightstats.ttest_ind(df.query("group == 'treatment'")['converted'],
                                                              df.query("group == 'control'")['converted'],
                                                              alternative="larger",
                                                              usevar='unequal')

In [ ]:
print("T-статистика={},  p_value={}".format(t_score, p_value))

#### Проведение вычислений. $Mann–Whitney$-test.

In [ ]:
w_score, p_value = scipy.stats.mannwhitneyu(df.query("group == 'treatment'")['converted'], 
                         df.query("group == 'control'")['converted'],
                         alternative="greater")

In [ ]:
print("W-статистика={},  p_value={}".format(w_score, p_value))

<a id='m12'></a>
### Ошибки первого и второго рода

|  | | | |
|----------|:---------|:--------|:---------|
|  | | 	Верная гипотеза: | |
|  |     | $H_0$   | $H_1$    |
| Результат применения критерия: | $H_0$   |$H_0$ верно принята  |$H_0$ неверно принята (ошибка II рода)  |  
|  |$H_1$   |$H_0$ неверно отвергнута(ошибка I рода)  | $H_0$ верно отвергнута   |



Уровень значимости (статистическая значимость, statistical significance) $\alpha$ - это и есть вероятность ошибки первого рода, т. е. вероятность принятия альтернативной гипотезы при условии, что на самом деле верна нулевая гипотеза. 

Обозначим  $\beta$  вероятность ошибки второго рода. 

Величина  $1-\beta$ называется *статистической мощностью* (statistical power) критерия. По сути мощность показывает, сколько значений, соответствующих альтернативной гипотезе, мы действительно отнесем к альтернативной гипотезе

![asd](https://habrastorage.org/files/475/9e5/ebc/4759e5ebcfc54b11a852704017d2d8ac.png)

![mem](https://i2.wp.com/flowingdata.com/wp-content/uploads/2014/05/Type-I-and-II-errors1.jpg?fit=960%2C720&ssl=1)